# Title 

Continuous Assessment 1 06/10/2023 - BD & ADA Module // MSc in Data Analytics Y1 S2 - <b>Student ID 2023063

## Introduction
write intro

--

Data source: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop/data

## Practical Big Data (PySparkSQL) - Prep data

In [1]:
#Import PySparkSQL
from pyspark.sql import SparkSession        

# Create SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQL")
  .getOrCreate())

# Path to dataset
csv_file = "file:////home/hduser/Downloads/CA1/2020-Jan.csv"

In [2]:
# Read and create a temporary view
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("rawdata")

In [3]:
# Visualise inferred schema
data = spark.sql("SELECT * FROM rawdata")
data.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [4]:
# Display 5 first rows
data.show(5)

+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|          event_time|event_type|product_id|        category_id|category_code|   brand|price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|2020-01-01 00:00:...|      view|   5809910|1602943681873052386|         null| grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:...|      view|   5812943|1487580012121948301|         null|kinetics| 3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:...|      view|   5798924|1783999068867920626|         null|  zinger| 3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:...|      view|   5793052|1487580005754995573|         null|    null| 4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:...|      view|   5899926|2115334439910245200|         null|    null| 3.92|484071203|cff70ddf-529e-4b0...|


In [5]:
# Number of users
action = spark.sql("SELECT count(distinct user_id) FROM rawdata")
action.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 410073|
+-----------------------+



In [6]:
#Number of events
action = spark.sql("SELECT count(distinct event_type) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT event_type)|
+--------------------------+
|                         4|
+--------------------------+



In [7]:
# Number of products
action = spark.sql("SELECT count(distinct product_id) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                     45484|
+--------------------------+



In [8]:
# Top seller product
countProduct = spark.sql("SELECT product_id, count(*) FROM rawdata GROUP BY product_id ORDER BY count(*) desc")
countProduct.show(10)

+----------+--------+
|product_id|count(1)|
+----------+--------+
|   5809910|   36967|
|   5809912|   14310|
|   5917178|   12634|
|   5700037|   11429|
|   5686925|   11098|
|   5854897|   11070|
|   5802432|    9865|
|   5751383|    9407|
|   5751422|    9365|
|   5815662|    9190|
+----------+--------+
only showing top 10 rows



In [9]:
# New dataframe with topseller product
topseller = spark.sql("SELECT DISTINCT * FROM rawdata WHERE product_id LIKE 5809910")
topseller.show(5)

+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|  brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 14:21:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|592285428|33ecf984-0438-459...|
|2020-01-01 16:56:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595651372|121148f9-4e42-48b...|
|2020-01-01 18:40:...|remove_from_cart|   5809910|1602943681873052386|         null|grattol| 5.24|583351843|85fa8bff-02e8-4a5...|
|2020-01-02 06:09:...|            view|   5809910|1602943681873052386|         null|gratto

In [10]:
# Add new column with condition to topseller df
# If event_time is in the first couple of weeks of the month, column FN1 (fortnight1) is equals to 1 (True)
# If in the last 2 weeks of the month column FN1 is equals 0 (False)

from pyspark.sql.functions import when
from pyspark.sql.functions import lit

topseller = topseller.withColumn("FN1", \
   when((topseller.event_time < '2020-01-16'), lit(1)) \
     .when((topseller.event_time >= '2020-01-15'), lit(0)) \
                                )

In [11]:
# Show dataframe
topseller.show(5)

+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+---+
|          event_time|      event_type|product_id|        category_id|category_code|  brand|price|  user_id|        user_session|FN1|
+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+---+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595414620|4adb70bb-edbd-498...|  1|
|2020-01-01 14:21:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|592285428|33ecf984-0438-459...|  1|
|2020-01-01 16:56:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595651372|121148f9-4e42-48b...|  1|
|2020-01-01 18:40:...|remove_from_cart|   5809910|1602943681873052386|         null|grattol| 5.24|583351843|85fa8bff-02e8-4a5...|  1|
|2020-01-02 06:09:...|            view|   5809910|160294368187

In [12]:
# Drop columns
topseller = topseller.drop(*["product_id","category_id","category_code","brand","price"])

In [13]:
# Show updated topseller df
topseller.show(100,False)

+-----------------------+----------------+---------+------------------------------------+---+
|event_time             |event_type      |user_id  |user_session                        |FN1|
+-----------------------+----------------+---------+------------------------------------+---+
|2020-01-01 00:00:00 UTC|view            |595414620|4adb70bb-edbd-4981-b60f-a05bfd32683a|1  |
|2020-01-01 14:21:02 UTC|view            |592285428|33ecf984-0438-459c-aea6-241319336d75|1  |
|2020-01-01 16:56:51 UTC|view            |595651372|121148f9-4e42-48ba-94eb-7aca8935940b|1  |
|2020-01-01 18:40:56 UTC|remove_from_cart|583351843|85fa8bff-02e8-4a5f-80b8-a129ce5555e3|1  |
|2020-01-02 06:09:01 UTC|view            |529378930|b99ea2eb-1c67-4f39-ab20-792ebfb87f93|1  |
|2020-01-02 08:21:31 UTC|view            |595842440|b257db78-93e8-46d4-9893-56194751a4d9|1  |
|2020-01-03 06:16:01 UTC|view            |595798798|8b2d9742-eb2f-4bbd-8387-ace81cfdef2a|1  |
|2020-01-03 11:17:22 UTC|cart            |596055322|74521846

In [33]:
from pyspark.sql.functions import asc
topseller.select("event_time","event_type","user_session").filter("user_id = 444458155").orderBy(asc("event_time")).show(truncate=False)

+-----------------------+----------------+------------------------------------+
|event_time             |event_type      |user_session                        |
+-----------------------+----------------+------------------------------------+
|2020-01-02 17:06:46 UTC|view            |34d0d7d2-07f0-4aa0-a4a9-07ff03e52306|
|2020-01-02 17:06:55 UTC|cart            |34d0d7d2-07f0-4aa0-a4a9-07ff03e52306|
|2020-01-02 17:12:47 UTC|cart            |bf38672f-ea97-4977-959e-7e459ac94f81|
|2020-01-02 17:15:01 UTC|remove_from_cart|bf38672f-ea97-4977-959e-7e459ac94f81|
|2020-01-02 17:15:05 UTC|remove_from_cart|bf38672f-ea97-4977-959e-7e459ac94f81|
|2020-01-02 17:46:04 UTC|view            |56604a11-65dd-4617-8a6b-a9ef637229ea|
|2020-01-02 17:46:12 UTC|cart            |56604a11-65dd-4617-8a6b-a9ef637229ea|
|2020-01-04 12:02:35 UTC|purchase        |205249b3-4ee4-4d5d-90df-cdfe539a5061|
|2020-01-07 17:57:09 UTC|view            |7b48418d-c227-4ac9-a1e1-e57ee16827c0|
+-----------------------+---------------